In [80]:
import pandas as pd
import numpy as np

In [82]:
def productivity(age, qol, debug=True):
    '''Calculates an individual's productivity, as the proportion of full working time,
    given their age and QoL'''
    
    prod_model = pd.read_csv("paid_prod_model.csv", header=None, index_col=0, names=["value"])
    # TODO - check all values read in and are floats
    
    MCS = (prod_model.loc["MCS_age_div_10"] * (age/10)) \
        + (prod_model.loc["MCS_EQ5D"] * qol) \
        + prod_model.loc["MCS_const"]

    PCS = (prod_model.loc["PCS_age_div_10"] * (age/10)) \
        + (prod_model.loc["PCS_EQ5D"] * qol) \
        + prod_model.loc["PCS_const"]
        
    prod_sum = ((prod_model.loc["prod_age_div_10"] * (age/10)) 
                + (prod_model.loc["prod_age_div_10_sq"] * ((age/10)**2)) 
                + (prod_model.loc["prod_MCS_div_10"] * (MCS/10)) 
                + (prod_model.loc["prod_MCS_div_10_sq"] * ((MCS/10)**2)) 
                + (prod_model.loc["prod_PCS_div_10"] * (PCS/10)) 
                + (prod_model.loc["prod_PCS_div_10_sq"] * ((PCS/10)**2)) 
                + prod_model.loc["prod_const"])
    
   
    return np.exp(prod_sum)/(1+np.exp(prod_sum))

In [83]:
productivity(51, 1)

value    0.703047
dtype: float64

In [ ]:
def paid_prod(age, gender, qol):
    '''Calculates paid production in £pcm, given individual's age, gender and QoL
    '''
    
    # First read in the on-costs estimate (everything else is calculated in called functions)
    
    try:
        with open("on_costs.csv", "r") as f:
            on_costs = f.read()
    except:
        print("Could not find on_costs.csv")
        return
      
    return productivity(age, qol) * wage(age, gender) * (1 + on_costs)